In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd 
import os
import cv2
import json
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU

os.makedirs('images', exist_ok=True)
os.makedirs('masks', exist_ok=True)
# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def create_masks_from_coco(coco_json_path, images_dir, masks_dir):
    with open(coco_json_path, 'r') as f:
        data = json.load(f)

    images_info = {img['id']: img for img in data['images']}
    annotations_by_image = {}
    for ann in data['annotations']:
        annotations_by_image.setdefault(ann['image_id'], []).append(ann)

    for img_id, info in images_info.items():
        filename = info['file_name']
        h, w = info['height'], info['width']
        mask = np.zeros((h, w), dtype=np.uint8)

        for ann in annotations_by_image.get(img_id, []):
            cat = int(ann['category_id'])
            for seg in ann['segmentation']:
                polygon = np.array(seg, dtype=np.int32).reshape(-1, 2)
                cv2.fillPoly(mask, [polygon], cat)

        mask_name = os.path.splitext(filename)[0] + '_mask.png'
        cv2.imwrite(os.path.join(masks_dir, mask_name), mask)

# Example usage:
# create_masks_from_coco('annotations.coco.json', 'images', 'masks')


In [ ]:
def create_masks_from_csv(csv_path, images_dir, masks_dir):
    df = pd.read_csv(csv_path)
    groups = df.groupby('image_id')

    for image_id, group in groups:
        img_path = os.path.join(images_dir, image_id)
        if not os.path.exists(img_path):
            continue
        img = cv2.imread(img_path)
        h, w = img.shape[:2]
        mask = np.zeros((h, w), dtype=np.uint8)

        for _, row in group.iterrows():
            seg = ast.literal_eval(row['segmentation'])
            pts = np.array(seg, dtype=np.int32).reshape(-1, 2)
            cv2.fillPoly(mask, [pts], color=int(row['category_id']))

        mask_name = os.path.splitext(image_id)[0] + '_mask.png'
        cv2.imwrite(os.path.join(masks_dir, mask_name), mask)

# Example usage:
# create_masks_from_csv('annotations.csv', 'images', 'masks')


In [ ]:
def load_data(img_dir, mask_dir, img_size=(256, 256)):
    X, Y = [], []
    for filename in os.listdir(img_dir):
        img_path = os.path.join(img_dir, filename)
        mask_path = os.path.join(mask_dir, os.path.splitext(filename)[0] + '_mask.png')
        if os.path.exists(mask_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, img_size)
            X.append(img)
            Y.append(mask)

    X, Y = np.array(X) / 255.0, np.expand_dims(np.array(Y), -1)
    return X, Y

X, Y = load_data('images', 'masks')
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)


In [ ]:
def unet(input_shape):
    inputs = layers.Input(input_shape)

    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D(2)(c1)

    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D(2)(c2)

    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(c3)

    u1 = layers.UpSampling2D(2)(c3)
    u1 = layers.concatenate([u1, c2])
    c4 = layers.Conv2D(128, 3, activation='relu', padding='same')(u1)

    u2 = layers.UpSampling2D(2)(c4)
    u2 = layers.concatenate([u2, c1])
    c5 = layers.Conv2D(64, 3, activation='relu', padding='same')(u2)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c5)

    model = models.Model(inputs, outputs)
    return model


In [ ]:
def unet_backbone(backbone_model, input_shape):
    inputs = layers.Input(shape=input_shape)
    backbone = backbone_model(include_top=False, input_tensor=inputs)

    skips = [backbone.get_layer(name).output for name in ['block_1_expand_relu', 'block_3_expand_relu', 'block_6_expand_relu', 'block_13_expand_relu']]
    output = backbone.output

    u = output
    for skip in reversed(skips):
        u = layers.UpSampling2D(2)(u)
        u = layers.concatenate([u, skip])
        u = layers.Conv2D(256, 3, activation='relu', padding='same')(u)

    u = layers.UpSampling2D(2)(u)
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(u)
    return models.Model(inputs, outputs)

mobilenet_unet = unet_backbone(MobileNetV2, (256, 256, 3))
resnet_unet = unet_backbone(ResNet50, (256, 256, 3))


In [ ]:
def compile_and_train(model, X_train, Y_train, X_val, Y_val, name):
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', MeanIoU(num_classes=2)])
    history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=8, epochs=20)
    model.save(f'{name}.h5')
    return history

unet_model = unet((256, 256, 3))
compile_and_train(unet_model, X_train, Y_train, X_val, Y_val, 'unet_model')
compile_and_train(mobilenet_unet, X_train, Y_train, X_val, Y_val, 'mobilenet_unet')
compile_and_train(resnet_unet, X_train, Y_train, X_val, Y_val, 'resnet_unet')


In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
